**Imports relevant packages**

In [ ]:
import datetime                    #for time data manipulation
import pandas as pd                #for data querying and processing
import dateutil.parser as dparser  #for extracting dates from string
from google.colab import files     #for downloading files from colab

import warnings                    #ignores warnings that clutter the output
warnings.filterwarnings("ignore")

**Reads GSFC clean windows .tab file and renames columns**

In [ ]:
df = pd.read_csv(r"/content/clean_windows_GSFC.tab",header=None,delimiter='\t',encoding='latin-1')
df = df.rename(columns={0: "Instrument_Number", 1: "Date_Received", 2: "Cleaned_Windows", 3: "Cleaning_Notes", 4: "Field_Ready", 5: "Field_Notes"})

**Removes certain characters in notes that negatively affect date extraction from string**

In [ ]:
for i in range(len(df)):
  if pd.isnull(df.at[i,'Cleaning_Notes']) == False:
    df['Cleaning_Notes'][i] = df['Cleaning_Notes'][i].replace("\x0b","")
    df['Cleaning_Notes'][i] = df['Cleaning_Notes'][i].replace(":","")
    df['Cleaning_Notes'][i] = df['Cleaning_Notes'][i].replace("-","")
  if pd.isnull(df.at[i,'Field_Notes']) == False:
    df['Field_Notes'][i] = df['Field_Notes'][i].replace("\x0b","")
    df['Field_Notes'][i] = df['Field_Notes'][i].replace(":","")
    df['Field_Notes'][i] = df['Field_Notes'][i].replace("-","")

**Replaces empty values in date columns with information extracted from notes columns**

In [ ]:
count_clean_dates_notna = len(df.loc[df['Cleaned_Windows'].notna()])
count_clean_dates_isna = len(df.loc[df['Cleaned_Windows'].isna()])
count_field_dates_notna = len(df.loc[df['Field_Ready'].notna()])
count_field_dates_isna = len(df.loc[df['Field_Ready'].isna()])

count_good_clean = 0
count_good_field = 0
count_notnan_clean = 0
count_notnan_field = 0

for i in range(len(df)):
  if pd.isnull(df.at[i,'Cleaned_Windows']) == True and pd.isnull(df.at[i,'Cleaning_Notes']) == False:
    count_notnan_clean = count_notnan_clean + 1
    try:
      df['Cleaned_Windows'][i] = dparser.parse(df['Cleaning_Notes'][i],fuzzy=True,dayfirst=True)
      count_good_clean = count_good_clean + 1
      df['Cleaned_Windows'][i] = datetime.datetime.strptime(df['Cleaned_Windows'][i], '%m/%d/%Y')
    except:
      ValueError

for i in range(len(df)):
  if pd.isnull(df.at[i,'Field_Ready']) == True and pd.isnull(df.at[i,'Field_Notes']) == False:
    count_notnan_field = count_notnan_field + 1
    try:
      df['Field_Ready'][i] = dparser.parse(df['Field_Notes'][i],fuzzy=True)
      count_good_field = count_good_field + 1
      df['Field_Ready'][i] = datetime.datetime.strptime(df['Field_Ready'][i], '%m/%d/%Y')
    except:
      ValueError

for i in range(len(df)):
  df['Date_Received'][i] = datetime.datetime.strptime(df['Date_Received'][i], '%m/%d/%Y').date()
  df['Cleaned_Windows'][i] = str(df['Cleaned_Windows'][i])[:10]
  df['Field_Ready'][i] = str(df['Field_Ready'][i])[:10]
  df['Cleaned_Windows'][i] = df['Cleaned_Windows'][i].replace("nan","")
  df['Field_Ready'][i] = df['Field_Ready'][i].replace("nan","")

**Appends summary information to a text file, then downloads it**

In [ ]:
with open(r'/content/Summary.txt' ,"w") as myFile:
  print("There were a total of",len(df),"records.",file=myFile)
  print("\nOut of the",len(df),"records,",count_clean_dates_notna,"had cleaning dates and",count_clean_dates_isna,"did not have cleaning dates.",file=myFile)
  print("Out of the",count_clean_dates_isna,"records with no cleaning dates,",count_clean_dates_isna - count_notnan_clean,"did not have cleaning notes and",count_notnan_clean,"had cleaning notes.",file=myFile)
  print("Out of the",count_notnan_clean,"records with cleaning notes,",count_good_clean,"of them successfully obtained date extractions, while",count_notnan_clean-count_good_clean,"of them failed due to string issues.",file=myFile)
  print("\nOut of the",len(df),"records,",count_field_dates_notna,"had field ready dates and",count_field_dates_isna,"did not have field ready dates.",file=myFile)
  print("Out of the",count_field_dates_isna,"records with no field ready dates,",count_field_dates_isna - count_notnan_field,"did not have field ready notes and",count_notnan_field,"had field ready notes.",file=myFile)
  print("Out of the",count_notnan_field,"records with field ready notes,",count_good_field,"of them successfully obtained date extractions, while",count_notnan_field-count_good_field,"of them failed due to string issues.",file=myFile)
  myFile.close()

df.to_csv('new_clean_windows_GSFC.tab', sep='\t', index=False,header=False)
files.download(r'/content/Summary.txt')
files.download(r'/content/new_clean_windows_GSFC.tab')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>